In [6]:
import requests
import json
from os import makedirs
from os.path import join, exists
from datetime import date, timedelta

In [7]:
API_ENDPOINT = 'https://newsapi.org/v2/everything'
with open('newsapi-org.txt', 'r') as f:
    API_KEY = f.read().strip()
    
DATA_DIR = join('data', 'ny_abc_articles')
makedirs(DATA_DIR, exist_ok=True)

In [8]:
API_KEY

'3d2142f3c3b44c2596d5912c319e4f0f'

In [35]:
search_params = {
    'from': '2017-09-01',
    'to': '2018-03-30',
    'sortBy': 'popularity',
    'pageSize': 100,
    'apiKey': API_KEY,
    'sources' : 'abc-news',
    'domains' : 'nytimes.com'
}

In [36]:
filename = join(DATA_DIR, 'ny_abc.json')
    
#if file not exists then only download it
pages_data = []
if not exists(filename):
    response = requests.get(API_ENDPOINT, search_params)
    data = response.json()
    results = data['articles']
    pages_data.extend(results)
        
    #write all data to json file
    with open(filename, 'w') as f:
        print("Writing to {} ...".format(filename))
        f.write(json.dumps(pages_data, indent=2))    

Writing to data/ny_abc_articles/ny_abc.json ...
